INTRODUCTION

In this tutorial I have created example code of using OpenAI and Huggingface model through LangChain library to create.
* LLM and chatmodels
* Prompt templates
* And output Parser

In [2]:
#import libraries
from langchain.llms import OpenAI
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.schema import HumanMessage,SystemMessage,AIMessage
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.chains import LLMChain

Using OpenAI model

In [3]:
# import OpenAI API key
os.environ["OPEN_API_KEY"]=""

In [4]:
# temperature has value from 0 to 1. vale close to 0 means roughly that the model will
# always select the highest probability word. And value close to 1 model might select a word with slightly lower probability. therby, leading to more variation, randomness and creativity.

llm=OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)
text="What are genes enriched in epilepsy"

print(llm.predict(text))

/Users/luqman/Documents/LangchainProject/LangChain-project/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/luqman/Documents/LangchainProject/LangChain-project/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




1. Sodium channel genes: Mutations in sodium channel genes, such as SCN1A, SCN2A, and SCN8A, have been linked to various forms of epilepsy, including Dravet syndrome, genetic epilepsy with febrile seizures plus, and early infantile epileptic encephalopathy.

2. Potassium channel genes: Mutations in potassium channel genes, such as KCNQ2 and KCNQ3, have been associated with benign familial neonatal seizures and early onset epileptic encephalopathy.

3. GABA receptor genes: GABA is the main inhibitory neurotransmitter in the brain and mutations in genes involved in GABA signaling, such as GABRA1, GABRG2, and GABRD, have been linked to different types of epilepsy, including generalized epilepsy with febrile seizures plus and juvenile myoclonic epilepsy.

4. Glutamate receptor genes: Glutamate is the main excitatory neurotransmitter in the brain and mutations in glutamate receptor genes, such as GRIN2A and GRIA1, have been linked to various forms of epilepsy, including focal epilepsy and

Using HuggingFace model

In [6]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=""
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})
output=llm_huggingface.predict("What are genes mutated in epilepsy")
print(output)

epilepsy


Note that OpenAI generate a more reasonable and comprehensive output that HuggingFace.
Letäs try a more simple question.

In [7]:
#OpenAI
text="What is the capital of Finland"

print(llm.predict(text))



The capital of Finland is Helsinki.


In [8]:
# HuggingFace
output=llm_huggingface.predict("What is the capital of Finland")
print(output)

helsinki


Both model generate correct answer but the answer generated by OpenAi is more comprehensive

Prompt Template and LLMchain

A prompt template typically refers to a predefined structure or format for providing instructions or queries to a language model. In the context of using models like GPT-3, a prompt is a piece of text that you input to get a response from the model. A prompt template, therefore, would be a standardized way of constructing these prompts to achieve desired results.

In [13]:
prompt_template=PromptTemplate(input_variables=['disease'],
template="What are the causes of {disease}")

prompt_template.format(disease="diabetes")

'What are the causes of diabetes'

In [14]:
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("diabetes"))



1. Genetics: Diabetes can be inherited from parents or other family members. People with a family history of diabetes are more likely to develop the condition.

2. Obesity: Being overweight or obese is a major risk factor for developing type 2 diabetes. Excess body fat, particularly in the abdomen, can make the body less sensitive to insulin.

3. Sedentary lifestyle: Lack of physical activity and a sedentary lifestyle can increase the risk of developing type 2 diabetes. Regular exercise helps to control weight and improve insulin sensitivity.

4. Unhealthy diet: Consuming a diet high in processed and sugary foods, and low in fruits and vegetables, can increase the risk of developing type 2 diabetes. These foods can cause spikes in blood sugar levels and contribute to insulin resistance.

5. Age: The risk of developing type 2 diabetes increases with age, particularly after the age of 45. This may be due to a decrease in physical activity and muscle mass, as well as changes in hormone 

Combining Multiple Chains Uing simple Sequential Chain

In [18]:
continent_template=PromptTemplate(input_variables=['continent'],
template="Tell me a common genetict disease in {continent}")

continent_chain=LLMChain(llm=llm,prompt=continent_template)

treat_template=PromptTemplate(input_variables=['disease'],
template="How to treat {disease}")
treat_chain=LLMChain(llm=llm,prompt=treat_template)

In [24]:
chain=SimpleSequentialChain(chains=[continent_chain,treat_chain])
chain.run("africa")

' While there is currently no cure for sickle cell disease, there are treatments and management strategies that can help improve quality of life and prevent complications.\n\n1. Medications\nThere are several medications that can help manage the symptoms of sickle cell disease. These include pain relievers, such as ibuprofen or acetaminophen, to help with the pain associated with sickle cell crises. Hydroxyurea is a medication that can help reduce the frequency of sickle cell crises and decrease the risk of other complications. Antibiotics may also be prescribed to help prevent infections.\n\n2. Blood transfusions\nIn severe cases of sickle cell disease, regular blood transfusions may be necessary. This involves receiving healthy red blood cells from a donor to replace the sickle-shaped cells in the body. Blood transfusions can help improve anemia and reduce the risk of stroke and other complications.\n\n3. Bone marrow transplant\nA bone marrow transplant is the only known cure for sic

Sequential Chain

In [25]:
continent_template=PromptTemplate(input_variables=['continent'],
template="Tell me a common genetic disease in {continent}")

continent_chain=LLMChain(llm=llm,prompt=continent_template, output_key="genetic disease")

treat_template=PromptTemplate(input_variables=['genetic disease'],
template="How to treat {genetic disease}")
treat_chain=LLMChain(llm=llm,prompt=treat_template, output_key="treat")

In [26]:
chain=SequentialChain(chains=[continent_chain,treat_chain],
input_variables=['continent'],
output_variables=['genetic disease',"treat"])

In [27]:
chain({'continent':"Africa"})

/Users/luqman/Documents/LangchainProject/LangChain-project/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'continent': 'Africa',
 'genetic disease': '\n\nSickle cell disease (SCD) is a common genetic disease in Africa. It is estimated that about 200,000 babies are born with SCD in Africa each year, making it the most prevalent genetic disorder on the continent. SCD is caused by a mutation in the gene that produces hemoglobin, the protein responsible for carrying oxygen in the blood. This mutation causes red blood cells to become sickle-shaped, making them less able to carry oxygen and causing them to get stuck in blood vessels, leading to pain, organ damage, and other complications. SCD is most commonly found in people of African descent, as well as in people from the Middle East, Mediterranean, and Indian subcontinent. ',
 'treat': '\n\nThere is currently no cure for SCD, but there are treatments that can help manage the symptoms and complications of the disease. These treatments include:\n\n1. Pain management: Pain is one of the most common symptoms of SCD, and it can be severe and debi

Chatmodels With ChatOpenAI

In [ ]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [ ]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

Prompt Template + LLM +Output Parsers

In [ ]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [ ]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)


])

In [ ]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})